In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import pandas as pd
import tensorflow as tf

In [ ]:
img_width=224
img_height=224
batch_size=32
num_of_classes=17
IMG_SIZE=224


training_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/train"
validation_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/val"
test_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/test"
all_data_directory = "/content/drive/MyDrive/Datasets/gemstone2/all"

In [ ]:
train_data = ImageDataGenerator( ##rescale = 1./255,
                                   horizontal_flip=True,
                                    vertical_flip = True,
                                   #validation_split=0.2
                                   )

validation_data = ImageDataGenerator ( ##rescale = 1./255,
                                      horizontal_flip=True,
                                      vertical_flip = True,
                                    )

test_data = ImageDataGenerator ( ##rescale = 1./255,
                                      horizontal_flip=True,
                                      vertical_flip = True,
                                    )


In [ ]:
training_gen = train_data.flow_from_directory(training_set_directory,
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle = True,
                                                 target_size = (img_height, img_width),
                                           # subset='training')
)

validation_gen = validation_data.flow_from_directory(validation_set_directory,
                                                     batch_size = batch_size,
                                                     class_mode = 'categorical',
                                                     shuffle = True,
                                                     target_size = (img_height, img_width),
                                                     #subset='validation')
)

test_gen = test_data.flow_from_directory(test_set_directory,
                                                     batch_size = batch_size,
                                                     class_mode = 'categorical',
                                                     shuffle = True,
                                                     target_size = (img_height, img_width),
                                                     #subset='validation')
)

Found 6542 images belonging to 70 classes.
Found 1857 images belonging to 70 classes.
Found 1000 images belonging to 70 classes.


In [ ]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.1),
        #layers.RandomTranslation(height_factor=0.25, width_factor=0.25),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = model.output
    x = layers.Conv2D(5120, kernel_size=2, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(70, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=70)

epochs = 100



In [ ]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
hist = model.fit(training_gen, epochs=epochs, validation_data=validation_gen, verbose=1, callbacks=[model_checkpoint_callback])

Epoch 1/100
205/205 [==============================] - 89s 220ms/step - loss: 2.3336 - accuracy: 0.4499 - val_loss: 2.0067 - val_accuracy: 0.5401
Epoch 2/100
205/205 [==============================] - 44s 212ms/step - loss: 1.2821 - accuracy: 0.6376 - val_loss: 1.4757 - val_accuracy: 0.6193
Epoch 3/100
205/205 [==============================] - 41s 198ms/step - loss: 0.9862 - accuracy: 0.7051 - val_loss: 1.4267 - val_accuracy: 0.6381
Epoch 4/100
205/205 [==============================] - 43s 210ms/step - loss: 0.8462 - accuracy: 0.7437 - val_loss: 1.5254 - val_accuracy: 0.6462
Epoch 5/100
205/205 [==============================] - 41s 200ms/step - loss: 0.7456 - accuracy: 0.7686 - val_loss: 1.3629 - val_accuracy: 0.6742
Epoch 6/100
205/205 [==============================] - 41s 200ms/step - loss: 0.6920 - accuracy: 0.7884 - val_loss: 1.3017 - val_accuracy: 0.6791
Epoch 7/100
205/205 [==============================] - 40s 192ms/step - loss: 0.6307 - accuracy: 0.8057 - val_loss: 1.4448 -

In [ ]:
checkpoint_filepath2 = '/tmp/checkpoint2'
model_checkpoint_callback2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.load_weights(checkpoint_filepath)

unfreeze_model(model)

epochs = 100
hist = model.fit(training_gen, epochs=epochs, validation_data=validation_gen, verbose=1, callbacks=[model_checkpoint_callback2])


In [ ]:
model.load_weights(checkpoint_filepath2)

In [ ]:
loss, accuracy = model.evaluate(test_gen)

19/19 [==============================] - 1s 57ms/step - loss: 0.9869 - accuracy: 0.8041
